This is not important for the task. We keep it just temporary for inspiration.

In [77]:
import os
import cv2
import numpy as np

folder= r"images"
image_array=[]

for filename in os.listdir(folder):
    input_path=os.path.join(folder, filename)
    image=cv2.imread(input_path)
    if image is None:
        print("image not found")
        continue
image_array.append(np.array(image))




KeyboardInterrupt: 

We have previosly created an csv file that contains the image_id and the answers to questions t01, t07 and t02 as weighted factors, for each image. See repository gz2_t01.csv. The code creates a new dataframe which labels every image to a main type (for question t01) according to the format used in the CustomImageDataset class.

In [199]:
import os
import cv2
import numpy as np
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from PIL import Image


img_dir=r"images"
csv_file_number=r"gz2_filename_mapping.csv"
csv_file_quest=r"gz2_hart16.csv"

file_number=pd.read_csv(csv_file_number)
file_quest=pd.read_csv(csv_file_quest)

file_merge = file_quest.merge(file_number, left_on='dr7objid', right_on='objid')[['objid', 'asset_id', 't01_smooth_or_features_a01_smooth_weighted_fraction', 't01_smooth_or_features_a02_features_or_disk_weighted_fraction', 't01_smooth_or_features_a03_star_or_artifact_weighted_fraction']]

print(file_merge)
file_merge.to_csv('gz2_merge.csv', index=False)


                     objid  asset_id  \
0       587732591714893851     58957   
1       588009368545984617    193641   
2       587732484359913515     55934   
3       587741723357282317    158501   
4       587738410866966577    110939   
...                    ...       ...   
239690  587741490371625059    262969   
239691  587731174917669027    275883   
239692  587731512070177108    282536   
239693  587731513145688256    284172   
239694  588015507679805636    288961   

        t01_smooth_or_features_a01_smooth_weighted_fraction  \
0                                                   0.000     
1                                                   0.024     
2                                                   0.780     
3                                                   0.036     
4                                                   0.767     
...                                                   ...     
239690                                              0.823     
239691         

In [200]:
img_dir=r"images"
csv_file=r"gz2_merge.csv"

file=pd.read_csv(csv_file)
file['asset_id'] = file['asset_id'].astype(str) + ".jpg"

for ind in file.index:
    input_path=os.path.join(img_dir, file['asset_id'][ind])
    photo=cv2.imread(input_path)
    if photo is None:
        file.drop(ind, inplace=True)

file.to_csv('existing_images.csv', index= False)
        

In [201]:
file.loc [file['t01_smooth_or_features_a01_smooth_weighted_fraction']>0.6, 't01_type'] = 0
file.loc [file['t01_smooth_or_features_a02_features_or_disk_weighted_fraction']>0.6, 't01_type'] = 1
file.loc [file['t01_smooth_or_features_a03_star_or_artifact_weighted_fraction']>=0.5, 't01_type'] = 2
 
file.dropna(inplace=True)
file.reset_index(drop=True, inplace=True)

t01_file=file[['asset_id','t01_type']].copy()

print(t01_file)
t01_file.to_csv('unbalanced_t01.csv',index=False)


          asset_id  t01_type
0        58957.jpg       1.0
1       193641.jpg       1.0
2        55934.jpg       0.0
3       158501.jpg       1.0
4       110939.jpg       0.0
...            ...       ...
209734   30593.jpg       0.0
209735   74398.jpg       0.0
209736  151540.jpg       0.0
209737  259387.jpg       0.0
209738  262969.jpg       0.0

[209739 rows x 2 columns]
